# Задание 2. SQL
## 2.1 Очень усердные ученики.

### 2.1.1 Условие

Образовательные курсы состоят из различных уроков, каждый из которых состоит из нескольких маленьких заданий. Каждое такое маленькое задание называется "горошиной".

Назовём очень усердным учеником того пользователя, который хотя бы раз за текущий месяц правильно решил 20 горошин.

### 2.1.2 Задача

Необходимо написать оптимальный запрос, который даст информацию о количестве очень усердных студентов.NB! Под усердным студентом мы понимаем студента, который правильно решил 20 задач за текущий месяц.

In [1]:
# Импорт библиотек
import pandahouse as ph

In [2]:
# создаем соединение с базой данных ClickHouse
connection1 = {'host': 'https://clickhouse.lab.karpov.courses',
                       'database': 'default',
                       'user': 'student', 
                       'password': 'dpo_python_2020'
               }

### Порядок действий
1. Вначале находим у каждой записи дату начала месяца и сразу через оконную функцию максимальную дату начала месяуа по все таблице<br>
2. Суммируем количество правильных ответов через оконку по каждому студенту и рабтаем только по текущему месяцу (st_month = cr_month)<br>
3. Подсчитываем уникальное количество студентов с горошинами более или равно 20.

In [3]:
# запрос SELECT
query1 = """
    select
      uniqExact(st_id) cnt_student
    from
      (
        select
          st_id,
          SUM(correct) OVER (PARTITION BY st_id) sum_sorrect
        from
          (
            select
              st_id,
              correct,
              toStartOfMonth(timest) st_month,
              MAX(toStartOfMonth(timest)) OVER() AS cr_month
            from
              default.peas
          ) q1
        where
          st_month = cr_month
      ) q2
    where
      sum_sorrect >= 20
"""

In [4]:
# Выполним запрос SELECT с помощью функции read_clickhouse и сохраним результат в объект DataFrame
df1 = ph.read_clickhouse(query1, connection=connection1)

# выводим результат
print(f"Количество усердных студентов: {df1['cnt_student'][0]}")

Количество усердных студентов: 136


## 2.2 Оптимизация воронки

### 2.2.1 Условие

Образовательная платформа предлагает пройти студентам курсы по модели trial: студент может решить бесплатно лишь 30 горошин в день. Для неограниченного количества заданий в определенной дисциплине студенту необходимо приобрести полный доступ. Команда провела эксперимент, где был протестирован новый экран оплаты.

### 2.2.2 Задача

Необходимо в одном запросе выгрузить следующую информацию о группах пользователей:

ARPU<br> 
ARPAU<br>
CR в покупку<br>
СR активного пользователя в покупку<br>
CR пользователя из активности по математике (subject = ’math’) в покупку курса по математике<br>

ARPU считается относительно всех пользователей, попавших в группы.

Активным считается пользователь, за все время решивший больше 10 задач правильно в любых дисциплинах.

Активным по математике считается пользователь, за все время решивший 2 или больше задач правильно по математике.

### Порядок действий


1. Создаем временную таблицу table_act (таблица активности).<cr>

    1.1. Вначале группируем по уникальному студенту сумму горошин и сумму горошин по математике. суммируем через оконки<cr>

    1.2. Далее определяем активность студента общую и по математике<cr>


2. Создаем временную таблицу table_sum (таблица денег). <cr>

    2.1. Вначале группируем по уникальному студенту сумму денег и сумму денег по математике. суммируем через оконки<cr>

    2.2. На верхнем уровне дополнительно к полям st_id, money_all, money_math формируем "флажки" оплаты<cr>

    
3. Основной запрос.<cr>

    3.1. Соединяем все три таблицы по st_id через LEFT JOIN.<cr>

    3.2. Группируем по test_grp и расчитываем указанные метрики. Если необходимо просуммировать через условие использую sumIf().

In [5]:
query2 = """
    WITH table_act AS (
      SELECT
        st_id,
        IF(sum_st > 10, 1, 0) active_all,
        IF(sum_st_math >= 2, 1, 0) active_math
      FROM
        (
          SELECT
            distinct st_id,
            SUM(correct) over(partition by st_id) sum_st,
            sumIf(correct, subject = 'Math') over(partition by st_id) sum_st_math
          FROM
            default.peas
        )
    ),
    table_sum AS (
      SELECT
        st_id,
        money_all,
        money_math,
        IF(money_all > 0, 1, 0) flag_all,
        IF(money_math > 0, 1, 0) flag_math
      FROM
        (
          SELECT
            distinct st_id,
            SUM(money) over (partition by st_id) money_all,
            sumIf(money, subject = 'Math') over(partition by st_id) money_math
          FROM
            default.final_project_check
        )
    )
    SELECT
      s.test_grp groups,
      ROUND(SUM(c.money_all) / count(s.st_id), 2) ARPU,
      ROUND(sumIf(c.money_all, a.active_all = 1) / sum(a.active_all), 2) ARPAU,
      ROUND(SUM(c.flag_all) / count(s.st_id)*100, 2) CR,
      ROUND(sumIf(c.flag_all, a.active_all = 1) / sum(a.active_all) * 100, 2) CR_Active,
      ROUND(sumIf(c.flag_math, a.active_math = 1) / sum(a.active_math) * 100, 2) CR_Active_Math
    FROM
      default.studs s
      LEFT JOIN table_act a ON s.st_id = a.st_id
      LEFT JOIN table_sum c ON s.st_id = c.st_id
    GROUP BY
      s.test_grp"""

In [6]:
# Выполним запрос SELECT с помощью функции read_clickhouse и сохраним результат в объект DataFrame
df2 = ph.read_clickhouse(query2, connection=connection1)

# выводим результат
print("Информация о группах пользователей:")

df2

Информация о группах пользователей:


,groups,ARPU,ARPAU,CR,CR_Active,CR_Active_Math
0,control,4540.98,10393.70,4.92,11.02,6.12
1,pilot,11508.47,29739.58,10.85,26.04,9.52
